# Perfilamiento sobre algoritmo Simplex

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2300.060
BogoMIPS:                        4600.19
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        512 KiB
L3 cache:                        45 MiB
NUMA node0 CPU(s):               0-3
Vul

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.2.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 16GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits


In [3]:
import math
import time
import Simplex as Simplex
import numpy as np
from pytest import approx
from scipy.optimize import linprog

**Módulo time**

Ejemplo Maximización para evualuar tiempo de ejecución

In [4]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
Simplex algorithm tomó 0.0018892288208007812 segundos


Con Scipy

In [5]:
c_scipu=[-3, -5]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Scipy  tomó 0.004729032516479492 segundos


Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

**Comando Magic %time**

In [6]:
%time problema.solve()

Optimization completed successfully !
Solution for x vector:
[0, 0]
Optimal value:
0
CPU times: user 727 µs, sys: 227 µs, total: 954 µs
Wall time: 610 µs


([0, 0], 0, 0)

**Cprofile**

Para poder visualizar en que secciones de código se tarda más

In [7]:
import cProfile

In [8]:
cprof = cProfile.Profile()
cprof.enable()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
         473 function calls in 0.002 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.002    0.001 interactiveshell.py:3400(run_code)
        3    0.000    0.000    0.002    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <ipython-input-8-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.002    0.002 Simplex.py:44(solve)
       16    0.000    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       10    0.000    0.000    0.000    0.000 iostream.py:384(write)
        1    0.000    0.000    0.000    0.000 index_tricks.py:317(__getitem__)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4  

In [9]:
import pstats

cprof.dump_stats("Simplex_stats")

In [10]:
p_simplex_stats = pstats.Stats("Simplex_stats")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

Sat May  8 02:59:10 2021    Simplex_stats

         473 function calls in 0.002 seconds

   Ordered by: cumulative time
   List reduced from 84 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.002    0.001 /home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400(run_code)
        3    0.000    0.000    0.002    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <ipython-input-8-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.002    0.002 /home/ubuntu/Simplex.py:44(solve)
       16    0.000    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       10    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/ipykernel/iostream.py:384(write)
        1    0.000    0.000    0.000    

In [11]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

Sat May  8 02:59:10 2021    Simplex_stats

         473 function calls in 0.002 seconds

   Ordered by: cumulative time
   List reduced from 84 to 7 due to restriction <'solve|module'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 <ipython-input-8-de806f07ed56>:4(<module>)
        1    0.000    0.000    0.002    0.002 /home/ubuntu/Simplex.py:44(solve)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:313(solve)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-de806f07ed56>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-de806f07ed56>:5(<module>)
        4    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:309(_solve_dispatcher)




In [12]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

473


In [13]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 84 to 10 due to restriction <10>

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3400(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       3    0.000    0.002  interactiveshell.py:3400(run_code)
<ipython-input-8-de806f07ed56>:4(<module>)                               <-       1    0.000    0.002  {built-in method builtins.exec}
Simplex.py:44(solve)                                                     <-       1    0.000    0.002  <ipython-input-8-de806f07ed56>:4(<module>)
{built-in method numpy.core._multiarray_umath.implement_array_function}  <-       1    0.000    0.000  <__array_function__ internals>:2(concatenate)
                                                                             

In [14]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

   Ordered by: cumulative time
   List reduced from 84 to 10 due to restriction <'print|dot|ndim|solve'>

Function                                 called...
                                             ncalls  tottime  cumtime
Simplex.py:44(solve)                     ->       1    0.000    0.000  <__array_function__ internals>:2(copy)
                                                  6    0.000    0.000  <__array_function__ internals>:2(dot)
                                                  2    0.000    0.000  <__array_function__ internals>:2(size)
                                                  4    0.000    0.000  <__array_function__ internals>:2(solve)
                                                  1    0.000    0.000  index_tricks.py:317(__getitem__)
                                                  2    0.000    0.000  twodim_base.py:156(eye)
                                                  7    0.000    0.000  {built-in method builtins.len}
                                

Obs. Se observa que las partes del código que tienen un mayor número de llamadas son

* **print**- El cual se utiliza al final del algoritmo para mostrar los valores óptimos 
* **dot**(producto punto)
* **dim** y **size** que se ocupan al principio del algoritmo para saber la longuitud de las matrices a optimizar
* **solve**- Este es el que tiene mayor número de llamadas

**Lineprofiler**

Para saber línea por línea en que parte nuestro algoritmo se está tardano más en ejecutar

In [15]:
import line_profiler

problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
([2.0, 6.0], -36.0, 0)


In [16]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.001334 s
File: /home/ubuntu/Simplex.py
Function: solve at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                               def solve(self):
    45                                                   
    46                                                   """
    47                                                   Solves the simplex algorithm. 
    48                                                   Returns
    49                                                   -------
    50                                                   :solution: Numpy array with solution
    51                                                   """
    52         1          3.0      3.0      0.2          problem = self.problem
    53         1          2.0      2.0      0.1          c_N = self.c
    54         1          2.0      2.0      0.1          A = self.A
    55         1          2.0      2.0      0.1        

De acuerdo a las estadísticas resultantes con line profiler se tienen 7 líneas en el código que tienen un porcentaje elevado de tiempo en ejecución:
* Línea 63 - `A = np.c_[A,identity_A]` que es la parte que construye la matriz completa A al agregar la identidad por las variables de holgura
* Línea 81 - `lista.append (-lambda_ + np.dot(nu, A[:, N_list_idx[i]]))` para crear la lista de las lambas a evaluar en el método
* Línea 88 - ` d = np.linalg.solve(B, A[:,idx_x_N]) ` para la solución del problema de ecuaciones
* Línea 97 -  `if np.isnan(lista2).all() == True:`para evular una solución *ounbounded*
* Línea 117 - `nu = np.linalg.solve(B.T, c_B)`solución del problema de ecuaciones
* Línea 137 - `print("Optimization completed successfully !") `para imprimir si la optimización fue correcta

In [ ]:
#Develop